# Determine the matrix dimension for a given number of matrix elements

In [1]:
import sys
sys.path.append('..')

In [2]:
import sparsity_pattern as spat
import math

In [3]:
import scipy.sparse
import numpy as np
from typing import List

def print_matrix(pat: List[List[int]], n_rows, n_cols):
    # convert to matrix
    mat = scipy.sparse.lil_matrix((n_rows, n_cols), dtype=np.int64)
    idx_rows, idx_cols = np.array(pat)[:, 0], np.array(pat)[:, 1]
    mat[idx_rows, idx_cols] = 1
    # print
    print("Sparsity Pattern:")
    print(pat)
    print("Matrix:")
    print(mat.todense())

# `'circle'` pattern
The matrix dimension `n` and the number of `offsets` determine the number of matrix elements for a quadratic sparse matrix with 'circle' pattern.
If the number of matrix elements `n_max_params` is given, the `get_matrix_dims_circle` function searches for hyperparameter combinations (n, offsets) that result in circle patterns with exactly or slightly less than `n_max_params` matrix elements.

### How to use `get_matrix_dims_circle`
The function `get_matrix_dims_circle` returns a list of dict.

In [4]:
n_max_params = 50
hyperparams = spat.utils.get_matrix_dims_circle(n_max_params)
hyperparams

[{'n': 50, 'offsets': [1]},
 {'n': 25, 'offsets': [1, 2]},
 {'n': 16, 'offsets': [1, 2, 3]},
 {'n': 12, 'offsets': [1, 2, 3, 4]},
 {'n': 10, 'offsets': [1, 2, 3, 4, 5]},
 {'n': 8, 'offsets': [1, 2, 3, 4, 5, 6]}]

Each dict is a feasible combination of a matrix dimension and offsets for the 'circle' pattern.

In [5]:
pat = spat.get("circle", **hyperparams[3])

n_dim = hyperparams[3]['n']
print_matrix(pat, n_dim, n_dim)

Sparsity Pattern:
[(0, 8), (0, 9), (0, 10), (0, 11), (1, 0), (1, 9), (1, 10), (1, 11), (2, 0), (2, 1), (2, 10), (2, 11), (3, 0), (3, 1), (3, 2), (3, 11), (4, 0), (4, 1), (4, 2), (4, 3), (5, 1), (5, 2), (5, 3), (5, 4), (6, 2), (6, 3), (6, 4), (6, 5), (7, 3), (7, 4), (7, 5), (7, 6), (8, 4), (8, 5), (8, 6), (8, 7), (9, 5), (9, 6), (9, 7), (9, 8), (10, 6), (10, 7), (10, 8), (10, 9), (11, 7), (11, 8), (11, 9), (11, 10)]
Matrix:
[[0 0 0 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 1 1 1]
 [1 1 0 0 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 0 0 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 0]]


### How does it work?
Assume we want to squeeze up to `n_max_params` weights into a quadratic sparse matrix with 'circle' pattern.

In [6]:
n_max_params = 50

The square root `int(sqr(n_max_params))` is the first smallest matrix dimension `n_min_dim` that can hold most of the desired `n_max_params` of weights. However, we need to subtract `-1` because the diagonals are assumed to be 0. `n_min_dim` is also the the maximum number of offsets we can squeeze into such a matrix.

In [7]:
n_min_dim = int(math.sqrt(n_max_params)) - 1

We loop over `n_offsets = [1, 2, .. n_min_dim]`.
The matrix dimension `n_dim` for the desired number of weights `n_max_params` and number of offsets `n_offsets` is `int(n_max_params / n_offsets)`.

In [8]:
for n_offsets in range(1, n_min_dim+1):
    n_dim = n_max_params // n_offsets    
    result = {"n_dim": n_dim, "offsets": list(range(1, n_offsets + 1))}
    
    # add more information
    if True:
        pat = spat.get("circle", n_dim, range(1, n_offsets + 1))
        n_act_params = len(pat)
        result = {**result, "n_act_params": n_act_params, "ratio_squeezed": n_act_params / n_max_params}
    print(result)

{'n_dim': 50, 'offsets': [1], 'n_act_params': 50, 'ratio_squeezed': 1.0}
{'n_dim': 25, 'offsets': [1, 2], 'n_act_params': 50, 'ratio_squeezed': 1.0}
{'n_dim': 16, 'offsets': [1, 2, 3], 'n_act_params': 48, 'ratio_squeezed': 0.96}
{'n_dim': 12, 'offsets': [1, 2, 3, 4], 'n_act_params': 48, 'ratio_squeezed': 0.96}
{'n_dim': 10, 'offsets': [1, 2, 3, 4, 5], 'n_act_params': 50, 'ratio_squeezed': 1.0}
{'n_dim': 8, 'offsets': [1, 2, 3, 4, 5, 6], 'n_act_params': 48, 'ratio_squeezed': 0.96}
